In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('CKD.csv')

In [3]:
dataset

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0.0,2.0,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3.0,0.0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [11]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [13]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [15]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=dataset['classification_yes']

In [17]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [18]:
# IMPORTANT name the X_test variable as X_test_ so that pre-standardized can be referred as X_test for actual vs. prediction comparison 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test_ = sc.transform(X_test)


In [19]:
from sklearn.tree import DecisionTreeClassifier

In [20]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [21]:
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['sqrt','log2'],
              'splitter':['best','random']} 



grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 
 



Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [27]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test_) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)




In [30]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'log2', 'splitter': 'random'}: 0.9701163285572423


In [32]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[51  0]
 [ 4 78]]


In [34]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

       False       0.93      1.00      0.96        51
        True       1.00      0.95      0.97        82

    accuracy                           0.97       133
   macro avg       0.96      0.98      0.97       133
weighted avg       0.97      0.97      0.97       133



In [36]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


C:\Anaconda3-AI-Training\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


0.4878048780487805

In [39]:
table=pd.DataFrame.from_dict(re)

In [41]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004629,0.002826,0.004932,0.001139,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.981378,0.905069,0.962573,0.922492,0.962573,0.946817,0.028375,5
1,0.003036,0.000913,0.005860,0.001466,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.962963,0.943699,0.981217,1.000000,1.000000,0.977576,0.021818,2
2,0.002838,0.001017,0.005437,0.000814,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.963284,0.905069,0.832663,0.962264,0.943651,0.921386,0.049112,8
3,0.002829,0.001007,0.004719,0.001083,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",1.000000,1.000000,0.962264,0.962264,0.981217,0.981149,0.016876,1
4,0.002616,0.000760,0.004827,0.001733,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.981569,0.923510,0.925146,0.962264,0.943093,0.947116,0.022238,4
5,0.002059,0.001354,0.004936,0.001000,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.926978,0.943041,0.925524,0.906705,0.981217,0.936693,0.025064,7
6,0.002559,0.001356,0.006010,0.001390,entropy,log2,best,"{'criterion': 'entropy', 'max_features': 'log2...",0.944707,0.962264,0.869925,0.981031,0.943093,0.940204,0.037739,6
7,0.002830,0.000988,0.006793,0.002380,entropy,log2,random,"{'criterion': 'entropy', 'max_features': 'log2...",0.944707,0.981014,0.925146,0.981031,0.981217,0.962623,0.023445,3


In [43]:
# Sample test values
test_data = pd.DataFrame({
    'age': [58],
    'bp': [73],
    'al': [0],
    'su': [0],
    'bgr': [76],
    'bu': [70],
    'sc': [1.988627554721604],
    'sod': [132.6570231874442],
    'pot': [4.52326998],
    'hrmo': [17.717169392290145],
    'pcv': [44],
    'wc': [5306],
    'rc': [5.194385103813284],
    'sg_b': [0],
    'sg_c': [0],
    'sg_d': [1],
    'sg_e': [0],
    'rbc_normal': [1],
    'pc_normal': [0],
    'pcc_present': [1],
    'ba_present': [1],
    'htn_yes': [0],
    'dm_yes': [0],
    'cad_yes': [1],
    'appet_yes': [1],
    'pe_yes': [0],
    'ane_yes': [1]
})

In [45]:
# Make predictions
#test_predictions = grid.predict(test_data)
#test_data['Predicted'] = test_predictions

Future_Prediction=grid.predict(test_data)# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[ True]


C:\Anaconda3-AI-Training\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [47]:
# Generate 100 random test samples
num_samples = 100

# Random test data generation
random_test_data = pd.DataFrame({
    'age': np.random.randint(20, 80, num_samples),
    'bp': np.random.randint(60, 180, num_samples),
    'al': np.random.randint(0, 5, num_samples),
    'su': np.random.randint(0, 5, num_samples),
    'bgr': np.random.randint(70, 300, num_samples),
    'bu': np.random.randint(10, 100, num_samples),
    'sc': np.random.uniform(0.5, 5.0, num_samples),
    'sod': np.random.uniform(130, 145, num_samples),
    'pot': np.random.uniform(3.0, 6.0, num_samples),
    'hrmo': np.random.uniform(8.0, 20.0, num_samples),
    'pcv': np.random.randint(30, 50, num_samples),
    'wc': np.random.randint(4000, 12000, num_samples),
    'rc': np.random.uniform(3.0, 5.5, num_samples),
    'sg_b': np.random.randint(0, 2, num_samples),
    'sg_c': np.random.randint(0, 2, num_samples),
    'sg_d': np.random.randint(0, 2, num_samples),
    'sg_e': np.random.randint(0, 2, num_samples),
    'rbc_normal': np.random.randint(0, 2, num_samples),
    'pc_normal': np.random.randint(0, 2, num_samples),
    'pcc_present': np.random.randint(0, 2, num_samples),
    'ba_present': np.random.randint(0, 2, num_samples),
    'htn_yes': np.random.randint(0, 2, num_samples),
    'dm_yes': np.random.randint(0, 2, num_samples),
    'cad_yes': np.random.randint(0, 2, num_samples),
    'appet_yes': np.random.randint(0, 2, num_samples),
    'pe_yes': np.random.randint(0, 2, num_samples),
    'ane_yes': np.random.randint(0, 2, num_samples)
})

# Make predictions
random_test_data['Predicted'] = grid.predict(random_test_data)

# Save to CSV for further analysis
random_test_data.to_csv('DC test_predictions.csv', index=False)
print("DC - Random test predictions saved to CSV.")

DC - Random test predictions saved to CSV.


C:\Anaconda3-AI-Training\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [49]:
# Initialize an empty DataFrame
table_pr = pd.DataFrame()

In [51]:
print(X_test)

           age     bp   al   su         bgr    bu   sc         sod       pot  \
132  46.000000   70.0  0.0  0.0  100.000000  47.0  0.5  142.000000  3.500000   
309  66.000000   90.0  2.0  0.0  153.000000  76.0  3.3  137.528754  4.627244   
334  69.000000   70.0  0.0  0.0   83.000000  42.0  1.2  139.000000  3.700000   
196  55.000000  100.0  1.0  4.0  297.000000  53.0  2.8  139.000000  4.500000   
246  60.000000   90.0  2.0  0.0  105.000000  53.0  2.3  136.000000  5.200000   
..         ...    ...  ...  ...         ...   ...  ...         ...       ...   
349  71.000000   70.0  0.0  0.0   81.000000  18.0  0.8  145.000000  5.000000   
168  50.000000   80.0  0.0  0.0  137.000000  46.0  0.8  139.000000  5.000000   
150  48.000000  100.0  0.0  0.0  103.000000  79.0  5.3  135.000000  6.300000   
392  51.492308   60.0  3.0  0.0  148.112676  34.0  1.2  137.528754  4.627244   
66   34.000000   60.0  0.0  0.0   91.000000  49.0  1.2  135.000000  4.500000   

          hrmo  ...  rbc_normal  pc_nor

In [53]:
# Add the specified columns to the DataFrame from X_test and other provided data
table_pr['age'] = X_test["age"]
table_pr['bp'] = X_test["bp"]  # Assuming "bp" is a column in X_test
table_pr['al'] = X_test["al"]  # Assuming "al" is a column in X_test
table_pr['su'] = X_test["su"]  # Assuming "su" is a column in X_test
table_pr['bgr'] = X_test["bgr"]  # Assuming "bgr" is a column in X_test
table_pr['bu'] = X_test["bu"]  # Assuming "bu" is a column in X_test
table_pr['sc'] = X_test["sc"]  # Assuming "sc" is a column in X_test
table_pr['sod'] = X_test["sod"]  # Assuming "sod" is a column in X_test
table_pr['pot'] = X_test["pot"]  # Assuming "pot" is a column in X_test
table_pr['hrmo'] = X_test["hrmo"]  # Assuming "hrmo" is a column in X_test
table_pr['pcv'] = X_test["pcv"]  # Assuming "pcv" is a column in X_test
table_pr['wc'] = X_test["wc"]  # Assuming "wc" is a column in X_test
table_pr['rc'] = X_test["rc"]  # Assuming "rc" is a column in X_test
table_pr['sg_b'] = X_test["sg_b"]  # Assuming "sg_b" is a column in X_test
table_pr['sg_c'] = X_test["sg_c"]  # Assuming "sg_c" is a column in X_test
table_pr['sg_d'] = X_test["sg_d"]  # Assuming "sg_d" is a column in X_test
table_pr['sg_e'] = X_test["sg_e"]  # Assuming "sg_e" is a column in X_test
table_pr['rbc_normal'] = X_test["rbc_normal"]  # Assuming "rbc_normal" is a column in X_test
table_pr['pc_normal'] = X_test["pc_normal"]  # Assuming "pc_normal" is a column in X_test
table_pr['pcc_present'] = X_test["pcc_present"]  # Assuming "pcc_present" is a column in X_test
table_pr['ba_present'] = X_test["ba_present"]  # Assuming "ba_present" is a column in X_test
table_pr['htn_yes'] = X_test["htn_yes"]  # Assuming "htn_yes" is a column in X_test
table_pr['dm_yes'] = X_test["dm_yes"]  # Assuming "dm_yes" is a column in X_test
table_pr['cad_yes'] = X_test["cad_yes"]  # Assuming "cad_yes" is a column in X_test
table_pr['appet_yes'] = X_test["appet_yes"]  # Assuming "appet_yes" is a column in X_test
table_pr['pe_yes'] = X_test["pe_yes"]  # Assuming "pe_yes" is a column in X_test
table_pr['ane_yes'] = X_test["ane_yes"]  # Assuming "ane_yes" is a column in X_test
table_pr['Actual']=y_test
table_pr['Predicted']=grid_predictions

In [55]:
#Exports the table_pr DataFrame to a CSV file to compare actual vs. predicted for the CKD.csv file values

table_pr.to_csv("DC-CKD Data Actual-Prediction.csv",index=False)
print("DC Actual vs Prediction saved to CSV.")

DC Actual vs Prediction saved to CSV.
